# LCA model for passenger air transport

## Project setup

In [ ]:
# Import libraries
import brightway2 as bw
import os 
import lca_algebraic as agb
from sympy import init_printing
import matplotlib.pyplot as plt

# Pretty print for Sympy
init_printing()

# Set current project
bw.projects.set_current('LCA_course')

# Init user database
USER_DB = 'Foreground Aircraft'
agb.resetDb(USER_DB)

## LCA model creation

In [ ]:
# Air transport activity from EcoInvent
passenger_transport_ecoinvent = agb.findTechAct("transport, passenger aircraft, long haul", loc='GLO')

# Kerosene combustion activity
combustion = agb.copyActivity(
    USER_DB,
    passenger_transport_ecoinvent,
    "kerosene combustion"
)
combustion.deleteExchanges("aircraft, passenger, long haul")
combustion.deleteExchanges("airport")
combustion.deleteExchanges("kerosene", single=False)
combustion.updateMeta(phase="combustion")

# Airport
airport = agb.findTechAct("market for airport")
airport = agb.copyActivity(
    USER_DB,
    airport,
    "market for airport"
)
airport.updateMeta(phase="airport")

# Aircraft production
aircraft_production = agb.findTechAct("market for aircraft, passenger, long haul")
aircraft_production = agb.copyActivity(
    USER_DB,
    aircraft_production,
    "market for aircraft production"
)
aircraft_production.updateMeta(phase="aircraft_production")

# Kerosene production
kerosene_production = agb.findTechAct("market for kerosene", loc="RoW")
kerosene_production = agb.copyActivity(
    USER_DB,
    kerosene_production,
    "market for kerosene production"
)
kerosene_production.updateMeta(phase="kerosene_production")

In [ ]:
# Fix for mismatch chemical formulas (until fixed by future brightway/lca-algebraic releases) 
# https://github.com/oie-mines-paristech/lca_algebraic/issues/44
for act in [combustion, kerosene_production]:
    for ex in act.exchanges():
            if "formula" in ex: 
                del ex["formula"]
                ex.save()

In [ ]:
# Air transport of one passenger kilometer
passenger_transport = agb.newActivity(
    USER_DB,
    "passenger transport, aircraft",
    "one passenger kilometer",
    {combustion: 1.0,
     airport: 8.88e-14,
     aircraft_production: 2.299e-11,
     kerosene_production: 0.026716876}
)

In [ ]:
# List of impact methods to consider
impact_method = agb.findMethods("climate change", mainCat="EF v3.1")[0]

# Compute impacts
agb.compute_impacts(
    
    # Activity/Process to assess
    [passenger_transport], 
    
    # list of impacts to consider
    [impact_method],

    # Split impacts to get contributions from each phase of the lifecycle
    axis="phase",
)

In [ ]:
# Validation
agb.compute_impacts(
    
    # Activity/Process to assess
    [passenger_transport_ecoinvent, passenger_transport], 
    
    # list of impacts to consider
    [impact_method], 
)

## Export model

In [ ]:
# Save database and parameters as Bzipped JSON
agb.export_db(USER_DB, "./db_aircraft.bw2")